In [1]:
pip install peft accelerate transformers datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 20.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nv

In [2]:
import os
import pandas as pd
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering, Trainer, TrainingArguments
import torch
from accelerate import Accelerator
#from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model
from transformers.data.data_collator import default_data_collator

# === CONFIGURATION ===
BASE_IMAGE_PATH = '/kaggle/input/dataset-curated-with-split-r'
CSV_PATH = os.path.join(BASE_IMAGE_PATH, 'Sf/Sf_qa_data_trimmed_train_r.csv')

# === ACCELERATOR INIT ===
accelerator = Accelerator()

# === LOAD BLIP MODEL & PROCESSOR ===
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base", use_fast=True)
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

# === LOAD YOUR CURATED CSV ===
df = pd.read_csv(CSV_PATH, header=None, names=['image_path', 'question', 'answer'])
print(f"Loaded custom dataset with {len(df)} entries.")

# Ensure proper types
df['answer'] = df['answer'].fillna('unknown').astype(str)
df['image_path'] = df['image_path'].astype(str)

# === TRAIN-TEST SPLIT ===
train_df = df #, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train size: {len(train_df)}")# | Test size: {len(test_df)}")

# === DEFINE CUSTOM DATASET ===
class VQADataset(torch.utils.data.Dataset):
    def __init__(self, df, processor, image_base_path):
        self.df = df
        self.processor = processor
        self.image_base_path = image_base_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        full_image_path = os.path.join(self.image_base_path, row['image_path'])

        try:
            image = Image.open(full_image_path).convert("RGB")
        except Exception as e:
            print(f"Failed to load {full_image_path}: {e}")
            image = Image.new("RGB", (224, 224), (0, 0, 0))  # Fallback image

        encoding = self.processor(
            images=image,
            text=row['question'],
            padding="max_length",
            max_length=128,
            truncation=True,
            return_tensors="pt",
            return_attention_mask=True
        )

        labels = self.processor.tokenizer(
            row['answer'],
            padding="max_length",
            truncation=True,
            max_length=32,
            return_tensors="pt"
        )["input_ids"]

        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        encoding["labels"] = labels.squeeze(0)
        return encoding

# === CREATE DATASET INSTANCE ===
train_dataset = VQADataset(train_df, processor, BASE_IMAGE_PATH)

# === APPLY LoRA TO MODEL ===
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)
print("LoRA applied.")

# === PREPARE MODEL FOR ACCELERATION ===
model = accelerator.prepare(model)

# === DEFINE TRAINING ARGUMENTS ===
training_args = TrainingArguments(
    output_dir="./results",
    run_name="blip_vqa_lora_finetune_curated",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
)

# === TRAINER SETUP ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=default_data_collator,
)

# === GPU INFO ===
if torch.cuda.is_available():
    print("GPU GPU Memory Usage Before Training:")
    print(torch.cuda.memory_summary())

# === START TRAINING ===
trainer.train()

# === SAVE MODEL ===
trainer.save_model("./blip_vqa_lora_r_16")
print("Model saved to './blip_vqa_lora_r_16'")


2025-05-13 20:58:19.534537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747169899.998253      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747169900.107908      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

GPU GPU Memory Usage Before Training:
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1479 MiB |   1479 MiB |   1479 MiB |      0 B   |
|       from large pool |   1468 MiB |   1468 MiB |   1468 MiB |      0 B   |
|       from small pool |     10 MiB |     10 MiB |     10 MiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |   1479 MiB |   1479 MiB |   1479 MiB |      0 B   |
|       from large pool | 

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
10,10.215400
20,10.030000
30,9.815300
40,9.615100
50,9.495000
60,9.359000
70,9.199800
80,9.087900
90,8.984200
100,8.887400


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Model saved to './blip_vqa_lora_r_16'
